In [ ]:
from google.colab import files
file_up = files.upload()

Saving chatgpt-java-main.zip to chatgpt-java-main.zip


In [31]:
from zipfile import ZipFile
file_name = "chatgpt-java-main.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [32]:
def evaluate_operation(operation, variable_dict):
  for var, value in variable_dict.items():
    operation = re.sub(r'\b' + re.escape(var) + r'\b', str(value), operation)
    # print(operation)
  try:
    return eval(operation)
  except:
    return operation

def parse_java_code(java_code):
  variable_dict = {}
  cont_dict = {}
  assignment_regex = r'(\w+)\s*=\s*([^<>]*)\s*;'
  array_regex = r'(\w+[\[\]]+)\s*=\s*([^<>=]*)\s*;'
  b_array_regex = r'\s*.*(\[\]\s*\w|\w+\[\]).*=(\s*.*);'
  unary_operator_handler = r'((\w)\s*([+\-*/%]))\s*=(.*)'
  clone_regex = r'=(.*\.(copy.*|clone.*)\((.*)\)).*'
  col_regex = r'\<.*\>\s*(\w+)=\s*(.*);'
  itr_regex = r'(\w+)\s*=\s*(\w+)\..*iterator.*'
  lines = java_code.split("\n")

  for line in lines:
    line = line.replace("++","+=1")
    line = line.replace("--","+=1")
    unary_matches = re.findall(unary_operator_handler, line,re.IGNORECASE|re.MULTILINE)
    if len(unary_matches):
      for found_match in unary_matches:
        # print(found_match)
        to_replace, var, operator, value = found_match
        line = re.sub(r'\b' + re.escape(to_replace), var, line)
        line = re.sub(r'\b' + re.escape(value)  , var+operator+value, line)
    # print(line)


    col_matches = re.findall(col_regex, line,re.IGNORECASE|re.MULTILINE)
    if len(col_matches):
      for cmatch in col_matches:
        # print(cmatch)
        variable, size = cmatch
        numbers = re.findall(r'\((\d+)\)',size)
        # print(numbers)
        if len(numbers):
          size = int(numbers[0])
          cont_dict[variable] = size
          variable_dict[variable+'.size()'] = size
        else:
          cont_dict[variable] = size

    clone_matches = re.findall(clone_regex, line,re.IGNORECASE|re.MULTILINE)
    if len(clone_matches):
      for cmatch in clone_matches:
        # print(cmatch)
        to_replace, func, value = cmatch
        numbers = re.findall(r'\d+',value)
        # print(numbers)
        if len(numbers):
          size = numbers[0]
          for i in range(1,len(numbers)):
            size = abs(numbers[i]-size)
          # print(size)
          line = re.sub(r'' + re.escape(to_replace), size, line)
          # print(line)
        else:
          line = re.sub(r'' + re.escape(to_replace), value, line)


    array_matches = re.findall(b_array_regex, line,re.IGNORECASE|re.MULTILINE)
    if len(array_matches):
      for amatch in array_matches:
        # print(amatch)
        variable, size = amatch
        numbers = re.findall(r'\[(\d+)\]',size)
        # print(numbers)
        if len(numbers):
          size = int(numbers[0])
          for i in range(1,len(numbers)):
            size = abs(int(numbers[i])*size)
          # print(size)
          variable = variable.replace("[","")
          variable = variable.replace("]","")
          cont_dict[variable] = size
          variable_dict[variable+'.length'] = size
        else:
          variable = variable.replace("[","")
          variable = variable.replace("]","")
          cont_dict[variable] = size
          variable_dict[variable+'.length'] = size


    assignment_match = re.findall(assignment_regex, line,re.IGNORECASE|re.MULTILINE)
    if len(assignment_match):
      variable_name, variable_value = assignment_match[0]
      # print(f"variable_name:{variable_name} variable_value:{variable_value}")

      evaluated_value = evaluate_operation(variable_value, variable_dict)
      if evaluated_value is not None:
        variable_dict[variable_name] = evaluated_value
      else:
        variable_dict[variable_name] = variable_value


    itr_matches = re.findall(itr_regex, line,re.IGNORECASE|re.MULTILINE)
    if len(itr_matches):
      for imatch in itr_matches:
        # print(imatch)
        variable, size = imatch
        try:
          size=cont_dict[size]
        except:
          try:
            size=variable_dict[size+".size()"]
          except:
            size =size

        variable_dict[variable] = size

    for var in variable_dict.keys():
      try:
        variable_dict[var] = evaluate_operation(variable_dict[var], variable_dict)
        variable_dict[var] = cont_dict[variable_dict[var]]
        variable_dict[var] = evaluate_operation(variable_dict[var], cont_dict)
      except:
        continue

  return variable_dict, cont_dict


In [33]:
global_dic = {}
parent_map = {}
child_map = {}
ult_score = 1

import re
def chk_if_closed(cur,end,split_content):
  chk=[]
  i = cur+1
  while i<end:
    if "{" in split_content[i]:
     chk.append("{")
    if "}" in split_content[i] and len(chk)==0:
      return True
    if "}" in split_content[i] and chk[-1]=="{":
      chk.pop()

    i+=1
  return False

# get function of the line (parent, entire content, time)
def extract_log_func(p,content,space_score, complex_score, ignore, ult_parent, store_score,level):
  global ult_score
  split_content = content.split("\n")
  # complex_score = 1
  p_info = {}
  parent_func_info = {}
  # space_score = 1
  for no, code in enumerate(split_content):
    log_func_matches = re.findall(r'\b'+re.escape(p)+'\s*\(', code, re.MULTILINE|re.IGNORECASE)
    if len(log_func_matches) and no!=ignore:
      for lmatch in log_func_matches:
        p_info[no] = get_line_state(split_content, no, level) #get complexity to execute p, parent of p
        parent = p_info[no]["parent_func"]
        complex_score += p_info[no]["complex_score"]*complex_score
        parent_map[ignore] = p_info[no]["parent_line"]
        try:
          child_map[p_info[no]["parent_line"]].append(ignore) #parent->child
        except:
          child_map[p_info[no]["parent_line"]]= []
          child_map[p_info[no]["parent_line"]].append(ignore)
        global_dic[ignore] = {"line":split_content[ignore],"space":space_score,"time":complex_score}
        ignore = p_info[no]["parent_line"]
        #recursion_check
        for par in parent:
          if p==par:
            try:
              print(" "*level+f"function {p} appears to be called recursively.Enter size of function arguments:")
              # print(split_content[no])
              args = p_info[no]["parent_args"].split(",")
              if len(args)>1:
                max_a = 1
                for a in args:
                  val = int(input(f"Enter size/value for: {a}"))
                  if(val>max_a):
                    max_a = val
                p_info[no]["complex_score"] = 2**max_a
                ult_score = ult_score+(store_score*p_info[no]["complex_score"])
                # print(complex_score)
              else:
                args = p_info[no]["parent_args"]
                val = int(input(f"Enter size/value for: {args}"))
                p_info[no]["complex_score"] = (2**val)
                ult_score = ult_score+(store_score*p_info[no]["complex_score"])
                # print(complex_score)
            except:
              args = p_info[no]["parent_args"]
              val = int(input(f"Enter size/value for: {args}"))
              p_info[no]["complex_score"] = (2**val)
              # print(complex_score)
              ult_score = ult_score+(store_score*p_info[no]["complex_score"])
            space_score = space_score+(space_score*p_info[no]["complex_score"])
          else:
            # print(space_score)
            # print(p_info[no]["complex_score"])
            space_score = space_score+(space_score*p_info[no]["complex_score"])
            complex_score = complex_score*p_info[no]["complex_score"]
            ignore = p_info[no]["parent_line"]
            if p == ult_parent:
              ult_score = ult_score+(store_score*p_info[no]["complex_score"])
              parent_func_info[no] = extract_log_func(par,content,space_score,complex_score,ignore,ult_parent,1,level)["log_func_info"]
            else:
              parent_func_info[no] = extract_log_func(par,content,space_score,complex_score,ignore,ult_parent,store_score*p_info[no]["complex_score"],level)["log_func_info"]
          # print(" "*level+f"{p} has complexity of " +str(p_info[no]["complex_score"]))
          # print(" "*level+f"Ult-score:{ult_score}  store_score: {store_score}")
  result={
        "complex_score":ult_score,
        "log_func_info": p_info,
        "parent_of_func_info": parent_func_info,
        "space_score": space_score,
        "ult_score": ult_score,
        "store_score": store_score
    }

  return result


#for each line
def get_line_state(split_content, line_no,level):
  parent_func = set()
  parent_args = set()
  score_metric = []
  to_spare = 0
  chk_passed = []
  need_input = []
  debug_help = {}
  entire_content =""
  cur = line_no
  start = 0
  for_while_chk = r'(for|while)\s*\((.+)\)'
  for_each_chk = r'(\w+)\..*(forEach)'
  stop = False
  while(cur>0 and not stop):
    cur = cur-1
    cur_line = split_content[cur]+split_content[cur+1]
    fun_chk = r'\s*(\w+)\s*=*\s*\((.*)[\)\-].*{'
    matches = re.findall(fun_chk, cur_line, re.MULTILINE|re.IGNORECASE)
    for fmatch in matches:
      fun, arg = fmatch
      if ";" in arg or ":" in arg or "&&" in arg or "||" in arg or "!=" in arg or "==" in arg:
        continue
      if "if" in cur_line or "for" in cur_line or "while" in cur_line or "forEach" in cur_line or "else" in cur_line or "else if" in cur_line or "catch" in cur_line or "stream" in cur_line:
        continue
      else:
        if len(arg.split(" ")) == 1 and len(arg)!=0:
          continue
        else:
          parent_func.add(fun)
          for a in arg.split(","):
            parent_args.add(a)
          debug_help[fun] = {"line":cur,"->":cur_line}
          start = cur
          stop = True
          break
  # print(f"Inside Function:{list(parent_func)}")
  cur = start
  while(cur<line_no):
    if cur==line_no-1 or ("{" in split_content[cur] or "{" in split_content[cur+1]):
      entire_content= entire_content +"\n"+ split_content[cur]
      cur_line = split_content[cur]
      parsed_variables, parsed_containers = parse_java_code(entire_content)
      # print(parsed_variables)
      # print(parsed_containers)
      # print(entire_content)
      fw_matches = re.findall(for_while_chk, cur_line,re.IGNORECASE|re.MULTILINE)
      # print(cur_line)
      if len(fw_matches):
        # print(f"For-while: {fw_matches}")
        chk_passed.append(cur)
        for imatch in fw_matches:
          # print(imatch)
          it, arg = imatch
          debug_help[cur] = {"line":f"{len(score_metric)}","->":cur_line}
          match_fnd = re.findall(r'(\w+)\s*:\s*(\w+)',arg,re.IGNORECASE|re.MULTILINE)
          for fnd in match_fnd:
            toset, var = fnd
            if var:
              try:
                score_metric.append(parsed_variables[var])
                parsed_variables[toset] = parsed_variables[var]
              except:
                try:
                  score_metric.append(parsed_containers[var])
                  parsed_variables[toset] = parsed_containers[var]
                except:
                  try:
                    score_metric.append(parsed_variables[var+".size()"])
                    parsed_variables[toset] = score_metric[-1]
                  except:
                    try:
                      score_metric.append(parsed_variables[var+".length"])
                      parsed_variables[toset] = score_metric[-1]
                    except:
                      score_metric.append(var)
                      parsed_variables[toset] = score_metric[-1]
                      need_input.append(cur_line)
              if(chk_if_closed(cur+1,line_no,split_content)):
                score_metric.pop()

          match_fnd = re.findall(r'\b(\w+)\s*(==|!=|>|<|>=|<=)\s*(\w+)\b',arg,re.IGNORECASE|re.MULTILINE)
          for fnd in match_fnd:
            toset,op, var = fnd
            if var:
              if var.isnumeric():
                try:
                  score_metric.append(abs(var - parsed_variables[toset]))
                  parsed_variables[toset] = score_metric[-1]
                except:
                  score_metric.append(var)
                  parsed_variables[toset] = score_metric[-1]
              else:
                try:
                  score_metric.append(parsed_variables[var])
                  parsed_variables[toset] = score_metric[-1]
                except:
                  score_metric.append(var)
                  parsed_variables[toset] = score_metric[-1]
                  need_input.append(cur_line)
              if(chk_if_closed(cur+1,line_no,split_content)):
                score_metric.pop()


      fe_matches = re.findall(for_each_chk, cur_line,re.IGNORECASE|re.MULTILINE)
      if len(fe_matches):
        debug_help[cur] = {"line":f"{len(score_metric)}","->":cur_line}
        chk_passed.append(cur)
        for imatch in fe_matches:
          # print(imatch)
          var, it = imatch
          try:
            score_metric.append(parsed_variables[var])
          except:
            try:
              score_metric.append(parsed_containers[var])
            except:
              try:
                score_metric.append(parsed_variables[var+".size()"])
              except:
                try:
                  score_metric.append(parsed_variables[var+".length"])
                except:
                  score_metric.append(var)
                  need_input.append(cur_line)
          if(chk_if_closed(cur+1,line_no,split_content)):
            score_metric.pop()

      if cur not in chk_passed:
        to_spare = to_spare + 1

    if "}" in split_content[cur]:
      to_spare = to_spare - 1
      # print(to_spare)
    while to_spare<0:
      score_metric.pop()
      to_spare = to_spare+1
    cur = cur+1

  replace_dic = {}
  score = 1
  for s in range(len(score_metric)):
    if not score_metric[s].isnumeric():
      try:
        score_metric[s] = replace_dic[score_metric[s]]
        score = score * score_metric[s]
      except:
        print(" "*level+f"Enter length of: {score_metric[s]}")
        replace_dic[score_metric[s]] = int(input())
        score_metric[s] = replace_dic[score_metric[s]]
        score = score * score_metric[s]
    else:
      score = score * score_metric[s]

  result = {
      "parent_func": parent_func, #parent-function of the line
      "parent_line": start, #parent-line of line
      "parent_args": parent_args,
      "score_metric": score_metric, #param-to-execute line
      # "debug_help": debug_help,
      "complex_score": score #score-to-execute-line
  }
  return result

#main-function runs on entire content of a java file
def extract_log_info(content, level):
  logline_catalogue = {}
  logline_index = [] #contains all log line numbers
  log_line_info = {} #contains parent function of log line, parent line
  total_score = {}
  complex_score = {}
  parent_func_info = {}
  split_content = content.split("\n")
  log_line_reg = r"(LOGGER|LOG|\w*)\.(INFO|debug|error|WARN\w*|log\w*|SEVERE|CONFIG|FINE\w*)(\(.*\))"
  # print(split_content)
  for line_no, line_code in enumerate(split_content):
    log_line_matches = re.findall(log_line_reg, line_code, re.MULTILINE|re.IGNORECASE) # when log-line is matched
    if len(log_line_matches):
      logline_index.append(line_no)
      for lmatch in log_line_matches:
        # print(" "*level+f"Inside line: {line_code}")
        variable, log_operation, log_content = lmatch
        logline_catalogue[line_no] = line_code #line no.->code
        log_line_info[line_no] = get_line_state(split_content, line_no,level) #get information regarding the log-line
        parent = log_line_info[line_no]["parent_func"] #parent of log-line
        ignore = log_line_info[line_no]["parent_line"]
        s_score = 1
        c_score = 1
        for p in parent:
          ult_parent = p
          global ult_score
          ult_score = 1
          parent_func_info[line_no] = extract_log_func(p,content,len(log_content),log_line_info[line_no]["complex_score"], ignore, ult_parent,1,level) #get information about parent function of log-line
          parent_map[line_no] = ignore #log-line->parent-line
          s_score = s_score * parent_func_info[line_no]["space_score"]
          c_score = c_score * parent_func_info[line_no]["complex_score"]
          try:
            child_map[ignore].append(line_no) #parent-line_>child-line
          except:
            child_map[ignore]= []
            child_map[ignore].append(line_no)
          global_dic[line_no] = {"line":split_content[line_no],"space":s_score,"time":c_score}
        total_score[line_no] = len(log_content) * log_line_info[line_no]["complex_score"] * c_score
        complex_score[line_no] = log_line_info[line_no]["complex_score"] * c_score
        # print(" "*level+f"{line_code} has a complexity of "+str(log_line_info[line_no]["complex_score"]))






  result = {
      "logline_catalogue": logline_catalogue,
      "logline_index": logline_index,
      # "log_line_info": log_line_info,
      # "space-score": space_score,
      "parent_func_info": parent_func_info,
      "total-score": total_score,
      "complex-score": complex_score
  }
  return result





In [36]:
java_files_dic = {}
java_files_list = []
import os
import re
import pandas as pd
def make_tree(path, level=0):
    tree = []
    java_files = False
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        if os.path.isfile(item_path) and item.endswith('.java'):
            java_files = True
            tree.append((level + 1, item))
            java_files_dic[item] = (os.path.join(path, item))
            java_files_list.append(item)
        elif os.path.isdir(item_path):
            sub_tree = make_tree(item_path, level + 1)
            if sub_tree:
                tree.append((level + 1, item))
                tree.extend(sub_tree)
                java_files = False
    # if java_files:
    #     tree.insert(0, (level, os.path.basename(path)))
    return tree

root_path = "chatgpt-java-main"
tree = make_tree(root_path)

for level, item in tree:
    print("  " * level + item)
    if item in java_files_list:
      print("  " * level +"File:", item)
      path = java_files_dic[item]
      with open(path, "r") as file:
        content = file.read()
        # print(content)
        comment_pattern = r"(?://[^\n]*|/\*(?:.|[\r\n])*?\*/|/\*\*(?:.|[\r\n])*?\*/)"
        comments = re.findall(comment_pattern, content)
        # print(comments)
        for comment in comments:
          content = content.replace(comment,"")
        # print(content)
        result = extract_log_info(content, level)
        score_data = []
        lines = []
        for i in result["logline_index"]:
          # print(result["logline_catalogue"][i])
          lines.append(result["logline_catalogue"][i])
          score_data.append(int(result["total-score"][i]))
          # print("total-score:"+str(result["total-score"][i]))
          # print("time-score:"+str(result["complex-score"][i]))
        data = {"line_no" : result["logline_index"],
                "code" : lines,
                "score": score_data}
        df = pd.DataFrame(data)
        df['Percentile Rank'] = df.score.rank(pct = True)*10
        if len(df):
          print("Log Report:")
          display(df)
        else:
          print("No Log Statements found")

  src
    main
      java
        com
          plexpt
            chatgpt
              ChatGPT.java
              File: ChatGPT.java
       function addInterceptor appears to be called recursively.Enter size of function arguments:
Enter size/value for: {'chain '}10
       function addInterceptor appears to be called recursively.Enter size of function arguments:
Enter size/value for: {'chain '}10
Log Report:


,line_no,code,score,Percentile Rank
0,87,"log.error(""请求异常：{}"", errorMsg);",21546,5.0
1,90,log.error(baseResponse.get...,38988,10.0


              util
                ChatContextHolder.java
                File: ChatContextHolder.java
No Log Statements found
                SseHelper.java
                File: SseHelper.java
No Log Statements found
                Proxys.java
                File: Proxys.java
No Log Statements found
              exception
                ChatException.java
                File: ChatException.java
No Log Statements found
              entity
                billing
                  CreditGrantsResponse.java
                  File: CreditGrantsResponse.java
No Log Statements found
                  Usage.java
                  File: Usage.java
No Log Statements found
                  Datum.java
                  File: Datum.java
No Log Statements found
                  Grants.java
                  File: Grants.java
No Log Statements found
                BaseResponse.java
                File: BaseResponse.java
No Log Statements found
                chat
                  ChatC

,line_no,code,score,Percentile Rank
0,96,"log.error(""请求出错：{}"", e);",14,10.0


              listener
                SseStreamListener.java
                File: SseStreamListener.java
No Log Statements found
                AbstractStreamListener.java
                File: AbstractStreamListener.java
        function onEvent appears to be called recursively.Enter size of function arguments:
Enter size/value for: {'EventSource eventSource', ' String id', ' String type', ' String data'}50
Log Report:


,line_no,code,score,Percentile Rank
0,53,"log.info(""Chat session completed: ...",48413695994232875,10.000000
1,82,"log.error(""Stream connection error...",42,7.142857
2,90,"log.error(""response：{}"", responseT...",29,5.714286
3,95,"log.error(""Chat session has be...",60,8.571429
4,96,"log.error(""检测到号被封了"");",11,1.428571
5,102,"log.error(""检测到官方超载了，赶紧优化你的代码，做...",26,4.285714
6,108,"log.warn(""onFailure error:{}"", e);",25,2.857143


                ConsoleStreamListener.java
                File: ConsoleStreamListener.java
No Log Statements found
              api
                Api.java
                File: Api.java
No Log Statements found
    test
      java
        com
          plexpt
            chatgpt
              Test.java
              File: Test.java
Log Report:


,line_no,code,score,Percentile Rank
0,38,"log.info(""余额：{}"", response.getTotalAva...",39,10.0


              StreamTest.java
              File: StreamTest.java
No Log Statements found


In [ ]:
# import os

# directory = "Java-Logging-Example-master"

# for filename in os.listdir(directory):
#     file_path = os.path.join(directory, filename)
#     print(file_path)
#     if os.path.isfile(file_path):
#         print(f"File: {filename}")
#     else:
#         print(f"Directory: {filename}")

In [ ]:
# import os

# def make_tree(path, level=0):
#     tree = []
#     for item in os.listdir(path):
#         item_path = os.path.join(path, item)
#         if os.path.isdir(item_path):
#             tree.append((level+1, item))
#             sub_tree = make_tree(item_path, level + 1)
#             tree.extend(sub_tree)
#         else:
#             tree.append((level, item))
#     return tree


# root_path = "Java-Logging-Example-master"
# tree = make_tree(root_path)


# for level, item in tree:
#     print("  " * level + item)


In [ ]:
# java_files_dic = {}
# java_files_list = []
# import os
# import re

# logger_regex = r'logger'
# log_regex = r'(LOGGER|LOG)(\.*?).(info|debug|error|warning)\((\"r?(.*?)\"(,|\+)(.*?)|\"(.*?)\")\)'
# log_func = r'(LOGGER|LOG)(\.*?).(info|debug|error|warning)\((.*?\(.*?\))\)'
# def make_tree(path, level=0):
#     tree = []
#     java_files = False
#     for item in os.listdir(path):
#         item_path = os.path.join(path, item)
#         if os.path.isfile(item_path) and item.endswith('.java'):
#             java_files = True
#             tree.append((level + 1, item))
#             java_files_dic[item] = (os.path.join(path, item))
#             java_files_list.append(item)
#         elif os.path.isdir(item_path):
#             sub_tree = make_tree(item_path, level + 1)
#             if sub_tree:
#                 tree.append((level + 1, item))
#                 tree.extend(sub_tree)
#                 java_files = False
#     # if java_files:
#     #     tree.insert(0, (level, os.path.basename(path)))
#     return tree



# root_path = "chatgpt-java-main"
# tree = make_tree(root_path)

# # For debug
# for level, item in tree:
#     print("  " * level + item)
#     if item in java_files_list:
#       print("  " * level +"File:", item)
#       path = java_files_dic[item]
#       with open(path, "r") as file:
#           chknxt = True
#           ctr = 0
#           control_structures = []
#           log_score = []
#           import_lines = []
#           inheritence_score = 0
#           local_pkg = []
#           nesting_lvl = 0
#           content = file.read()
#           code_lines = content.split("\n");
#           for line in code_lines:
#             if not chknxt and ctr == 1:
#               chknxt = True
#               ctr = 0
#             elif not chknxt and ctr == 0:
#               ctr = 1
#             if "import" in line:
#               for j in java_files_list:
#                 find = j.split(".java")
#                 if find[0] in line:
#                   local_pkg.append(find[0])
#               inheritence_score+=1
#               import_lines.append(line)
#             if "if" in line and "{" in line:
#               control_structures.append("if")
#               log_score.append(1)
#               chknxt = True
#             elif "if" in line and "{" in code_lines[code_lines.index(line)+1]:
#               control_structures.append("if")
#               log_score.append(1)
#               chknxt = False
#             elif "else" in line and "{" in line:
#               control_structures.append("else")
#               log_score.append(1)
#               chknxt = True
#             elif "else" in line and "{" in code_lines[code_lines.index(line)+1]:
#               control_structures.append("else")
#               log_score.append(1)
#               chknxt = False
#             elif "for" in line and "{" in line:
#               control_structures.append("for")
#               log_score.append(int(input("Enter tentative number of time that this loop:\n"+line+"will execute:")))
#               chknxt = True
#             elif "for" in line and "{" in code_lines[code_lines.index(line)+1]:
#               control_structures.append("for")
#               log_score.append(int(input("Enter tentative number of time that this loop:\n"+line+code_lines[code_lines.index(line)+1]+"will execute:")))
#               chknxt = False
#             elif "try" in line and "{" in line:
#               control_structures.append("try")
#               log_score.append(1)
#               chknxt = True
#             elif "try" in line and  "{" in code_lines[code_lines.index(line)+1]:
#               control_structures.append("try")
#               log_score.append(1)
#               chknxt = False
#             elif "catch" in line and "{" in line:
#               control_structures.append("catch")
#               log_score.append(1)
#               chknxt = True
#             elif "catch" in line and "{" in code_lines[code_lines.index(line)+1]:
#               control_structures.append("catch")
#               log_score.append(1)
#               chknxt = False
#             elif "switch" in line and "{" in line:
#               control_structures.append("switch")
#               log_score.append(1)
#               chknxt = True
#             elif "switch" in line and "{" in code_lines[code_lines.index(line)+1]:
#               control_structures.append("switch")
#               log_score.append(1)
#               chknxt = False
#             elif "while" in line and "{" in line:
#               control_structures.append("while")
#               log_score.append(int(input("Enter tentative number of time that this loop:\n"+line+"will execute:")))
#               chknxt = True
#             elif "while" in line and "{" in code_lines[code_lines.index(line)+1]:
#               control_structures.append("while")
#               log_score.append(int(input("Enter tentative number of time that this loop:\n"+line+code_lines[code_lines.index(line)+1]+"will execute:")))
#               chknxt = False
#             elif "while" in line and "}" in line:
#               control_structures.append("do")
#               log_score.append(int(input("Enter tentative number of time that this loop:\n"+line+"will execute:")))
#               chknxt = True
#             elif "while" in line and "}" in code_lines[code_lines.index(line)-1]:
#               control_structures.append("do while")
#               log_score.append(int(input("Enter tentative number of time that this loop:\n"+line+code_lines[code_lines.index(line)+1]+"will execute:")))
#               chknxt = False
#             elif "public" in line and "{" in line:
#               control_structures.append("class/function")
#               log_score.append(1)
#               chknxt = True
#             elif "public" in line and "{" in code_lines[code_lines.index(line)+1]:
#               control_structures.append("class/function")
#               log_score.append(1)
#               chknxt = False
#             elif "private" in line and "{" in line:
#               control_structures.append("class/function")
#               log_score.append(1)
#               chknxt = True
#             elif "private" in line and  "{" in code_lines[code_lines.index(line)+1]:
#               control_structures.append("class/function")
#               log_score.append(1)
#               chknxt = False
#             elif "protected" in line and "{" in line:
#               control_structures.append("class/function")
#               log_score.append(1)
#               chknxt = True
#             elif "protected" in line and "{" in code_lines[code_lines.index(line)+1]:
#               control_structures.append("class/function")
#               log_score.append(1)
#               chknxt = False
#             elif chknxt and "{" in line:
#               control_structures.append("function")
#               log_score.append(1)
#               chknxt = True
#             if "{" in line:
#               nesting_lvl+=1
#             if "}" in line:
#               nesting_lvl-=1
#               try:
#                 control_structures.pop()
#                 log_score.pop()
#               except:
#                 print("")
#             logger_search =  re.findall(logger_regex, line, re.MULTILINE|re.IGNORECASE)
#             if(len(logger_search)>0):
#               for statement in logger_search:
#                 print("  " * level +"LOGGER DEFINITION:", line)
#                 print("  " * level +"=============================")
#             log_statements = re.findall(log_func, line, re.MULTILINE|re.IGNORECASE)
#             if(len(log_statements)>0):
#               for statement in log_statements:
#                 print("  " * level +"Log code:", line)
#                 print("  " * level +"-----------------------------")
#                 while len(control_structures) < nesting_lvl:
#                   chk = code_lines[temp_idx-1]
#                   temp_idx = temp_idx - 1
#                   control_structures.append(chk.split(" ")[0])
#                 print("  " * level +"Control structures:", control_structures)
#                 print("  " * level +"Loop value:", log_score)
#                 final_score = 1
#                 for score in log_score:
#                   final_score = final_score*score
#                 print("  " * level +"Log score:", final_score)
#                 # print("  " * level +"Inheritence number:", inheritence_score)
#                 # print("  " * level +"Nesting Level:", nesting_lvl)
#                 print("  " * level +"Log Level:", statement[2])
#                 if statement[3]:
#                   print("  " * level +"Function called:", statement[3])
#                 print("  " * level +"-----------------------------")
#             else:
#               log_statements = re.findall(log_regex, line, re.MULTILINE|re.IGNORECASE)
#               for statement in log_statements:
#                 print("  " * level +"Log code:", line)
#                 print("  " * level +"-----------------------------")
#                 temp_idx = code_lines.index(line)
#                 while len(control_structures) < nesting_lvl:
#                   chk = code_lines[temp_idx-1]
#                   temp_idx = temp_idx - 1
#                   control_structures.append(chk.split(" ")[0])

#                 print("  " * level +"Control structures:", control_structures)
#                 print("  " * level +"Loop value:", log_score)
#                 final_score = 1
#                 for score in log_score:
#                   final_score = final_score*score
#                 print("  " * level +"Log score:", final_score)
#                 # print("  " * level +"Local Package Inherited:", local_pkg)
#                 # print("  " * level +"Inheritence number:", inheritence_score)
#                 # print("  " * level +"Nesting Level:", nesting_lvl)
#                 print("  " * level +"Log Level:", statement[2])
#                 if statement[7]:
#                   print("  " * level +"Message:", statement[7])
#                 elif statement[4]:
#                   print("  " * level +"Message:", statement[4])
#                   if statement[6]:
#                     print("  " * level +"Variable:", statement[6])
#                 print("  " * level +"-----------------------------")
#           print("  " * level +"Number of lines:", len(code_lines))


# print(java_files_list)


  src
    test
      java
        com
          plexpt
            chatgpt
              Test.java
              File: Test.java
Enter tentative number of time that this loop:
    @Before    public void before() {will execute:10
Enter tentative number of time that this loop:
    public void before() {will execute:10
              Log code:         log.info("余额：{}", response.getTotalAvailable());
              -----------------------------
              Control structures: ['class/function', 'for', 'for']
              Loop value: [1, 10, 10]
              Log score: 100
              Log Level: info
              Function called: "余额：{}", response.getTotalAvailable()
              -----------------------------
              Number of lines: 65
              StreamTest.java
              File: StreamTest.java
Enter tentative number of time that this loop:
    @Before    public void before() {will execute:10
Enter tentative number of time that this loop:
    public void before() {will ex